# <div align="center"> 平台 API 测试</div>

In [1]:
from k12libs.utils.nb_easy import k12ai_post_platform_request
from k12libs.utils.nb_easy import k12ai_get_data
from k12libs.utils.nb_easy import k12ai_del_data
from k12libs.utils.nb_easy import k12ai_print

## 1. 平台状态`/k12ai/platform/stats`


A. 输入

```json
{
    "op":"", // [M] api具有子集操作, 如: "query"
    "user": "", // [M] 用户ID, api调用者, 如: 用户账号user_id(不含有特殊字符)
    "service_name": "", // [M] 服务名, k12ai由多个子框架组成, 需要指定具体服务, 如: "k12nlp", "k12cv"
    "service_uuid": "", // [M] 服务唯一标识码, 可以是用户项目的工程ID, project_id.
    "async": true,  // [O] 默认同步, 即值为false, api调用过程方式
    "service_params": { // [M] 服务参数
        "cpus": true, // [O] k12ai主机cpu信息 
        "gpus": true, // [O] k12ai主机gpu信息
        "disks": false, // [O] k12ai主机磁盘信息
        "containers": true // [O] k12ai主机上正在运行的label为"k12ai.service.name"的container信息
    }
}
```

B. 异步输出

```json
{
    "version": "0.1.0", // [M] 返回json格式的版本号, 兼容版本时使用
    "type": "result",   // [M] 消息体类型
    "tag": "platform",  // [M] 标识输出结果的服务TAG, 可以用来后续消息过滤 
    "op": "stats",      // [M] 标识输出结果由哪个功能产出, 一般统一为"/k12ai/{{tag}}/{{op}}"的形式
    "user": "",         // [M] 用户ID
    "service_uuid": "", // [M] 服务唯一标识码
    "timestamp": , // [M]
    "datetime": , // [M]
    "result": { // [M] 输出结果的正文内容
        "code": 100000,
        "descr": "task service success",
        "message": {
            "cpu_percent": 47.4,  // [C] k12ai主机cpu使用率, 不区分多cpu 
            "cpu_percent_list": [ // [C] k12ai主机cpu使用率, 每个cpu的使用率
                63.6,
                31.1
            ],
            "cpu_memory_total":, // [C] k12ai主机memory总大小(Byte)
            "cpu_memory_usage":, // [C] k12ai主机memory已使用(Byte)
            "cpu_memory_percent": , // [C] k12ai主机memory使用率
            "gpus": [  // [C] k12ai主机gpu信息, 每个gpu信息列表
                {
                    "name": "", // [C] gpu名字, 厂商
                    "gpu_percent": , // [C] gpu使用率
                    "gpu_memory_total": , // [C] gpu显存总大小(Byte)
                    "gpu_memory_usage": , // [C] gpu显存已使用(Byte)
                    "gpu_memory_percent": // [C] gpu显存使用率
                }
            ],
            "containers": [ // k12ai主机正在运行的关于训练相关的container信息
                {
                    "id": "", // container的ID
                    "cpu_percent": , // container的cpu使用率
                    "cpu_memory_total":,  // container的memory总大小
                    "cpu_memory_usage":,  // container的memory已使用
                    "cpu_memory_percent":, // container的memory利用率
                    "op": "",  // container正在执行的操作,如:"train.start", "predict.start"等
                    "user": "", // container用户ID
                    "service_uuid": "" // container执行的任务/服务ID.
                }
            ]
        }
    }
}
```

C. 同步输出:

```json
{
    "code": 100000,
    "descr": "task service success",
    "message": {
        "cpu_percent": 47.4,  // [C] k12ai主机cpu使用率, 不区分多cpu 
        "cpu_percent_list": [ // [C] k12ai主机cpu使用率, 每个cpu的使用率
            63.6,
            31.1
        ],
        "cpu_memory_total":, // [C] k12ai主机memory总大小(Byte)
        "cpu_memory_usage":, // [C] k12ai主机memory已使用(Byte)
        "cpu_memory_percent": , // [C] k12ai主机memory使用率
        "gpus": [  // [C] k12ai主机gpu信息, 每个gpu信息列表
            {
                "name": "", // [C] gpu名字, 厂商
                "gpu_percent": , // [C] gpu使用率
                "gpu_memory_total": , // [C] gpu显存总大小(Byte)
                "gpu_memory_usage": , // [C] gpu显存已使用(Byte)
                "gpu_memory_percent": // [C] gpu显存使用率
            }
        ],
        "containers": [ // k12ai主机正在运行的关于训练相关的container信息
            {
                "id": "", // container的ID
                "cpu_percent": , // container的cpu使用率
                "cpu_memory_total":,  // container的memory总大小
                "cpu_memory_usage":,  // container的memory已使用
                "cpu_memory_percent":, // container的memory利用率
                "op": "",  // container正在执行的操作,如:"train.start", "predict.start"等
                "user": "", // container用户ID
                "service_uuid": "" // container执行的任务/服务ID.
            }
        ]
    }
}
```

In [2]:
isasync = True
query = '''{
    "cpus": true,
    "gpus": true,
    "disks": false,
    "containers": true
}'''

stats = k12ai_post_platform_request(
    uri='k12ai/platform/stats',
    op='query',
    user='test',
    uuid='uuid',
    params=query,
    isasync=isasync
)
print("async input:")
k12ai_print(stats['req'])
if isasync:
    k12ai_del_data(stats['key'])
    print('async out:')
    k12ai_print(k12ai_get_data(stats['key'], 'result', waitcnt=5))

async input:
{
    "op": "query",
    "user": "test",
    "service_name": "k12platform",
    "service_uuid": "uuid",
    "service_params": {
        "cpus": true,
        "gpus": true,
        "disks": false,
        "containers": true
    },
    "async": true
}
async out:
[
    {
        "key": "platform/test/uuid/query/result",
        "value": {
            "version": "0.1.0",
            "type": "result",
            "tag": "platform",
            "user": "test",
            "service_uuid": "uuid",
            "op": "query",
            "timestamp": 1575891161593,
            "datetime": "2019-12-09 19:32:41",
            "result": {
                "code": 100000,
                "descr": "success",
                "message": {
                    "cpu_percent": 14.1,
                    "cpu_percent_list": [
                        14.8,
                        15.2,
                        11.3,
                        11.1,
                        13.7,
                        

In [9]:
isasync = False
query = '''{
    "cpus": true,
    "gpus": true,
    "disks": false,
    "containers": true
}'''

stats = k12ai_post_platform_request(
    uri='k12ai/platform/stats',
    op='query',
    user='test',
    uuid='uuid',
    params=query,
    isasync=isasync
)
print('sync input:')
k12ai_print(stats['req'])
print('sync output:')
k12ai_print(stats['res'])

sync input:
{
    "op": "query",
    "user": "test",
    "service_name": "k12platform",
    "service_uuid": "uuid",
    "service_params": {
        "cpus": true,
        "gpus": true,
        "disks": false,
        "containers": true
    },
    "async": false
}
sync output:
{
    "code": 100000,
    "descr": "success",
    "message": {
        "cpu_percent": 13.2,
        "cpu_percent_list": [
            15.3,
            13.7,
            9.9,
            15.1,
            12.6,
            12.7,
            12.3,
            13.3
        ],
        "cpu_memory_total": 50648887296,
        "cpu_memory_usage": 3733827584,
        "cpu_memory_percent": 8.8,
        "gpus": [
            {
                "name": "Tesla P40",
                "gpu_percent": 29.0,
                "gpu_memory_total": 22919.0,
                "gpu_memory_usage": 1121.0,
                "gpu_memory_percent": 4.89
            }
        ],
        "containers": [
            {
                "id": "8b6ffc639

## 2. 平台控制/k12ai/platform/control

A. 输入

```json
{
    "op": "",  // [M] 控制命令, 如, 停止某个container任务"container.stop"
    "user": "", // [M] 用户ID, api调用者, 如: 用户账号user_id(不含有特殊字符)
    "service_name": "", // [M] 服务名, k12ai由多个子框架组成, 需要指定具体服务, 如: "k12nlp", "k12cv"
    "service_uuid": "", // [M] 服务唯一标识码, 可以是用户项目的工程ID, project_id.
    "async": true, // [O] 控制命令的是否异步操作, 提醒: 有些命令操作是耗时的, 需要采用异步方式
    "service_params": { // [M] 服务参数
    }
}
```

B. 输出

```json
{
    "code": 100000,
    "descr": "task service success",
    "message": "container id f3726eae19d6 stop success"
}
```

### 2.1 停止container

In [4]:
result = k12ai_get_data(stats['key'], 'result')
try:
    cid = result[0]['value']['result']['containers'][0]['id']
except Exception:
    cid = 'unknow'

isasync = False
stats = k12ai_post_platform_request(
    uri='k12ai/platform/control',
    op='container.stop',
    user='test',
    uuid='uuid',
    params='{"id":"%s"}' % cid,
    isasync=isasync
)
k12ai_print(stats)
if isasync:
    k12ai_del_data(stats['key'])
    print('async out:')
    k12ai_print(k12ai_get_data(stats['key'], 'result'))

{
    "req": {
        "op": "container.stop",
        "user": "test",
        "service_name": "k12platform",
        "service_uuid": "uuid",
        "service_params": {
            "id": "unknow"
        },
        "async": false
    },
    "res": {
        "code": 100202,
        "descr": "task service start fails",
        "message": "container id unknow is not found"
    },
    "key": "platform/test/uuid/container.stop"
}
